# Mapping with Altair

This notebook covers progressive mapping using Altair. It accompanies the Vega-lite examples.

1. **Country Boundaries** - Using a `geo` mark.
2. **Local Authority Districts** .
3. **Regional Economic Indicators** - Using a transform to link data and map.


In [1]:
# Import required libraries
import altair as alt
import pandas as pd
from vega_datasets import data

# Enable Altair to render in Jupyter
alt.data_transformers.enable('json')

# Set base path for data files
BASE_PATH = ""

## 1. Country Boundaries

Start with basic country-level maps showing Scotland and Wales boundaries. These provide geographic context and demonstrate simple TopoJSON loading.

In [3]:
# Scotland country boundary
scotland_map = alt.Chart(
    alt.topo_feature(BASE_PATH + "geo_files/scotland.geojson", 'features')
).mark_geoshape(
    fill='lightblue',
    stroke='white',
    strokeWidth=1
).properties(
    title='Scotland',
    width=400,
    height=500
).project(
    type='mercator'
)

scotland_map

alt.Chart(...)

In [4]:
# Wales country boundary
wales_map = alt.Chart(
    alt.topo_feature(BASE_PATH + "geo_files/wales.geojson", 'features')
).mark_geoshape(
    fill='lightgreen',
    stroke='white',
    strokeWidth=1
).properties(
    title='Wales',
    width=400,
    height=500
).project(
    type='mercator'
)

wales_map

alt.Chart(...)

## 2. Local Authority Districts

Move to more detailed administrative boundaries and create choropleth maps. This demonstrates:
- Loading TopoJSON data with specific feature names
- Data joining between CSV metrics and geographic boundaries
- Color encoding for quantitative data

In [5]:
# Scotland Local Authority Districts - Base Map
scotland_lad_base = alt.Chart(
    alt.topo_feature(BASE_PATH + "geo_data/LAD_SCT_2025_05.topojson", 'object_name')
).mark_geoshape(
    fill='lightblue',
    stroke='white',
    strokeWidth=0.5
).properties(
    title='Scotland Local Authorities',
    width=400,
    height=500
).project(
    type='mercator'
)

scotland_lad_base

alt.Chart(...)

In [6]:
# Load wages data for choropleth mapping
# Note: We filter to 2024 data and use lookup transform for geographic join
scotland_choropleth = alt.Chart(
    BASE_PATH + "series_data/wages_LAD_SCT.csv"
).transform_filter(
    alt.datum.date == 2024
).transform_lookup(
    lookup='geo_id',
    from_=alt.LookupData(
        data=alt.topo_feature(BASE_PATH + "geo_data/LAD_SCT_2025_05.topojson", 'object_name'),
        key='properties.LAD25CD'
    ),
    as_='geo'
).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    shape=alt.Shape('geo:G'),
    color=alt.Color(
        'value:Q',
        scale=alt.Scale(scheme='viridis'),
        title='Wages (£)'
    ),
    tooltip=[
        alt.Tooltip('geo.properties.LAD25NM:N', title='Local Authority'),
        alt.Tooltip('value:Q', title='Wages (£)', format=',.0f'),
        alt.Tooltip('date:O', title='Year')
    ]
).properties(
    title='Scotland Wages by Local Authority (2024)',
    width=400,
    height=500
).project(
    type='mercator'
)

scotland_choropleth

alt.Chart(...)

In [7]:
# Wales choropleth using similar pattern but different color scheme
wales_choropleth = alt.Chart(
    BASE_PATH + "series_data/wages_LAD_WLS.csv"
).transform_filter(
    alt.datum.date == 2024
).transform_lookup(
    lookup='geo_id',
    from_=alt.LookupData(
        data=alt.topo_feature(BASE_PATH + "geo_data/LAD_WLS_2025_05.topojson", 'object_name'),
        key='properties.LAD25CD'
    ),
    as_='geo'
).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    shape=alt.Shape('geo:G'),
    color=alt.Color(
        'value:Q',
        scale=alt.Scale(scheme='plasma'),
        title='Wages (£)'
    ),
    tooltip=[
        alt.Tooltip('geo.properties.LAD25NM:N', title='Local Authority'),
        alt.Tooltip('value:Q', title='Wages (£)', format=',.0f'),
        alt.Tooltip('date:O', title='Year')
    ]
).properties(
    title='Wales Wages by Local Authority (2024)',
    width=400,
    height=500
).project(
    type='mercator'
)

wales_choropleth

alt.Chart(...)

## 3. Regional Economic Indicators

Create interactive maps with temporal controls. This demonstrates:
- **Layered visualization** with basemap + choropleth
- **Interactive parameters** for time series exploration
- **Series-first data flow** for optimal performance (load CSV data first, then lookup geography)
- **Light grey basemap** to show context for areas without data

In [8]:
# Scotland Interactive Wages Over Time
# Uses layered approach: grey basemap + colored choropleth

# Create year selection parameter
year_selector = alt.binding_range(min=2020, max=2024, step=1, name='Year: ')
year_param = alt.param(bind=year_selector, value=2024, name='year_select')

# Base layer - light grey background showing all LADs
scotland_base_layer = alt.Chart(
    alt.topo_feature(BASE_PATH + "geo_data/LAD_SCT_2025_05.topojson", 'object_name')
).mark_geoshape(
    fill='lightgrey',
    stroke='white',
    strokeWidth=0.5
)

# Data layer - choropleth with time filtering
scotland_data_layer = alt.Chart(
    BASE_PATH + "series_data/wages_LAD_SCT.csv"
).add_params(
    year_param
).transform_filter(
    alt.expr.datum.date == year_param
).transform_lookup(
    lookup='geo_id',
    from_=alt.LookupData(
        data=alt.topo_feature(BASE_PATH + "geo_data/LAD_SCT_2025_05.topojson", 'object_name'),
        key='properties.LAD25CD'
    ),
    as_='geo'
).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    shape=alt.Shape('geo:G'),
    color=alt.Color(
        'value:Q',
        scale=alt.Scale(scheme='viridis'),
        title='Wages (£)'
    ),
    tooltip=[
        alt.Tooltip('geo.properties.LAD25NM:N', title='Local Authority'),
        alt.Tooltip('value:Q', title='Wages (£)', format=',.0f'),
        alt.Tooltip('date:O', title='Year')
    ]
)

# Combine layers
scotland_interactive = alt.layer(
    scotland_base_layer,
    scotland_data_layer
).properties(
    title='Scotland Wages Over Time',
    width=400,
    height=500
).project(
    type='mercator'
)

scotland_interactive

AttributeError: type object 'expr' has no attribute 'datum'

In [9]:
# Wales Interactive Wages Over Time
# Similar pattern but separate parameter instance

year_selector_wales = alt.binding_range(min=2020, max=2024, step=1, name='Year: ')
year_param_wales = alt.param(bind=year_selector_wales, value=2024, name='year_select_wales')

wales_base_layer = alt.Chart(
    alt.topo_feature(BASE_PATH + "geo_data/LAD_WLS_2025_05.topojson", 'object_name')
).mark_geoshape(
    fill='lightgrey',
    stroke='white',
    strokeWidth=0.5
)

wales_data_layer = alt.Chart(
    BASE_PATH + "series_data/wages_LAD_WLS.csv"
).add_params(
    year_param_wales
).transform_filter(
    alt.expr.datum.date == year_param_wales
).transform_lookup(
    lookup='geo_id',
    from_=alt.LookupData(
        data=alt.topo_feature(BASE_PATH + "geo_data/LAD_WLS_2025_05.topojson", 'object_name'),
        key='properties.LAD25CD'
    ),
    as_='geo'
).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    shape=alt.Shape('geo:G'),
    color=alt.Color(
        'value:Q',
        scale=alt.Scale(scheme='plasma'),
        title='Wages (£)'
    ),
    tooltip=[
        alt.Tooltip('geo.properties.LAD25NM:N', title='Local Authority'),
        alt.Tooltip('value:Q', title='Wages (£)', format=',.0f'),
        alt.Tooltip('date:O', title='Year')
    ]
)

wales_interactive = alt.layer(
    wales_base_layer,
    wales_data_layer
).properties(
    title='Wales Wages Over Time',
    width=400,
    height=500
).project(
    type='mercator'
)

wales_interactive

AttributeError: type object 'expr' has no attribute 'datum'

## 4. Detailed Geographic Analysis - MSOA Level

Moving beyond Local Authority Districts to Middle Layer Super Output Areas (MSOAs) provides much finer geographic detail. This demonstrates:
- **Higher resolution mapping** with smaller administrative areas
- **Population density analysis** using census data
- **Fine-grained spatial patterns** not visible at LAD level

In [10]:
# Wales Population Density at MSOA Level
# Uses the simplified MSOA boundaries and population density data

wales_msoa_pop_density = alt.Chart(
    BASE_PATH + "series_data/wales_pop_density_msoa.csv"
).transform_lookup(
    lookup='geo_id',
    from_=alt.LookupData(
        data=alt.topo_feature(BASE_PATH + "geo_data/MSOA_WLS_2021.topojson", 'data'),
        key='properties.MSOA21CD'
    ),
    as_='geo'
).mark_geoshape(
    stroke='white',
    strokeWidth=0.3
).encode(
    shape=alt.Shape('geo:G'),
    color=alt.Color(
        'value:Q',
        scale=alt.Scale(scheme='oranges', type='log'),
        title='Population per km²'
    ),
    tooltip=[
        alt.Tooltip('geo.properties.MSOA21NM:N', title='MSOA Name'),
        alt.Tooltip('value:Q', title='Pop. Density (per km²)', format='.1f')
    ]
).properties(
    title='Wales Population Density by MSOA (2021)',
    width=500,
    height=600
).project(
    type='mercator'
)

wales_msoa_pop_density

alt.Chart(...)

## Summary

This notebook demonstrates a progression from simple geographic visualization to complex interactive analysis:

1. **Basic Maps**: Simple boundary visualization using GeoJSON data
2. **Choropleth Maps**: Data-driven color encoding with TopoJSON efficiency and CSV data joins
3. **Interactive Analysis**: Time-series exploration with layered basemaps and user controls
4. **Detailed Geographic Analysis**: Fine-grained MSOA-level population patterns

**Key Technical Patterns:**
- **Data Flow**: Series-first approach (CSV → geography lookup) for optimal performance
- **Layering**: Grey basemap + colored data layer for context
- **Transforms**: Filter → Lookup → Encode pipeline for data preparation
- **Interactivity**: Parameters with UI bindings for user exploration
- **Multi-scale Analysis**: From country level to MSOA level for different analytical needs

The progression shows how geographic visualization can evolve from static display to interactive analytical tools.